In [17]:
from pathlib import Path
import re
from dataclasses import dataclass
import srt

In [40]:
AutelSrtPath:Path = Path("./SampleData/MAX_0001.srt")
DjiSrtPath:Path = Path("./SampleData/DJI_0173.srt")

In [3]:
# Read all content and parse the srt content
with open(AutelSrtPath,"rt") as autelSrtFile:
    autelSrtString = autelSrtFile.read()

In [49]:
AutelSrtGenerator = srt.parse(autelSrtString)

In [5]:
Autel_content = AutelSrtGenerator.__next__()

In [8]:
Autel_content.content

'HOME(E: 121.498550, N: 24.601643) 2024-08-02 13:24:37\nGPS(E: 121.498672, N: 24.601662, 441.18m) \nISO:100 SHUTTER:30 EV:0.0 F-NUM:2.8 \nF.PRY (0.4°, -1.9°, 171.1°), G.PRY (0.0°, 0.0°, 171.2°)'

## DJI subtitle example
Mini2    
```
1
00:00:00,000 --> 00:00:01,000
F/2.8, SS 1667.11, ISO 100, EV 0, DZOOM 1.000, GPS (120.3534, 23.1201, 26), D 499.16m, H 98.00m, H.S 4.81m/s, V.S 0.10m/s 
```

Online:  
"""
1
00:00:01,000 --> 00:00:02,000
HOME(149.0251,-20.2532) 2017.08.05 14:11:51
GPS(149.0251,-20.2533,16) BAROMETER:1.9
ISO:100 Shutter:60 EV: Fnum:2.2
"""

pattern [GPS](https://pythex.org/?regex=GPS%5C((%3FP%3CEW%3E%5BEW%5D)%3A%5Cs%2B(%3FP%3CLon%3E%5Cd%2B%5C.%5Cd%2B)%5C%2C%5Cs%2B(%3FP%3CNS%3E%5BNS%5D)%3A%5Cs%2B(%3FP%3CLat%3E%5Cd%2B%5C.%5Cd%2B)%2C%5Cs%2B(%3FP%3CAlt%3E%5Cd%2B.%5Cd%2B)m%5C)&test_string=%27HOME(E%3A%20121.498550%2C%20N%3A%2024.601643)%202024-08-02%2013%3A24%3A37%0AGPS(E%3A%20121.498672%2C%20N%3A%2024.601662%2C%20441.18m)%20%0AISO%3A100%20SHUTTER%3A30%20EV%3A0.0%20F-NUM%3A2.8%20%0AF.PRY%20(0.4°%2C%20-1.9°%2C%20171.1°)%2C%20G.PRY%20(0.0°%2C%200.0°%2C%20171.2°)%27&ignorecase=0&multiline=0&dotall=0&verbose=0)

DJI Subtitle GPS alt represents?

In [20]:
# re patterns
GPS_pattern = re.compile(r"GPS\((?P<ew>[EW]):\s+(?P<lon>\d+\.\d+)\,\s+(?P<ns>[NS]):\s+(?P<lat>\d+\.\d+),\s+(?P<alt>\d+.\d+)m\)")
Camera_pattern = re.compile(r"ISO:(?P<iso>\d+)\s+SHUTTER:(?P<shutter>\d+\.?\d*)\s+EV:(?P<ev>\d+\.\d+)\s+F\-NUM:(?P<f_num>\d+\.\d+)")

In [26]:
GPS_match = re.search(GPS_pattern,Autel_content.content)
GPS_match_dict = GPS_match.groupdict()

In [22]:
print(GPS_match.groupdict())

{'ew': 'E', 'lon': '121.498672', 'ns': 'N', 'lat': '24.601662', 'alt': '441.18'}


In [27]:
Camera_match = re.search(Camera_pattern,Autel_content.content)
Camera_match_dict = Camera_match.groupdict()

In [19]:
print(Camera_match.groupdict())

{'iso': '100', 'shutter': '30', 'ev': '0.0', 'f_num': '2.8'}


In [35]:
@dataclass
class DjiMini2SrtMessage():
    """
    Sample massage:
    F/2.8, SS 1667.11, ISO 100, EV 0, DZOOM 1.000, GPS (120.3534, 23.1201, 26), D 499.16m, H 98.00m, H.S 4.81m/s, V.S 0.10m/s 
    """
    f_step:float = 0. #aperture 
    ss:float = 0. # shutter speed
    iso:int = 0
    ev:float = 0. 
    dzoom:float = 0.
    gps_lon:float = 0.
    gps_lat:float = 0.
    gps_sats:int = 0
    home_distance:float = 0.
    altitude:float = 0.
    horizontal_speed:float = 0.
    vertical_speed:float = 0.
    def toSrtMessage(self):
        """Export to SRT message."""
        return f"""F/{str(self.f_step)}, SS {str(self.ss)}, ISO {str(self.iso)}, EV {str(self.ev)}, DZOOM {str(self.dzoom)}, GPS ({str(self.gps_lon)}, {str(self.gps_lat)}, {str(self.gps_sats)}), D {str(self.home_distance)}m, H {str(self.altitude)}m, H.S {str(self.horizontal_speed)}m/s, V.S {str(self.vertical_speed)}m/s """

In [28]:
gps_lon=0.
gps_lat=0.
if GPS_match_dict["ew"] == "W":
    gps_lon = -float(GPS_match_dict["lon"])
else:
    gps_lon = float(GPS_match_dict["lon"])
if GPS_match_dict["ns"] == "S":
    gps_lat = -float(GPS_match_dict["lat"])
else:
    gps_lat = float(GPS_match_dict["lat"])
print(gps_lon,gps_lat)

121.498672 24.601662


In [32]:
djiFormatMsg = DjiMini2SrtMessage(
    f_step=float(Camera_match_dict["f_num"]),
    ss=float(Camera_match_dict["shutter"]),
    iso=int(Camera_match_dict["iso"]),
    ev=float(Camera_match_dict["ev"]),
    gps_lon=gps_lon,
    gps_lat=gps_lat,
    altitude=float(GPS_match_dict["alt"])   
)

In [33]:
djiFormatMsg.toSrtMessage()

'F/2.8, SS 30.0, ISO 100, EV 0.0, DZOOM 0.0, GPS (121.498672, 24.601662, 0), D 0.0m, H 441.18m, H.S 0.0m/s, V.S 0.0m/s '

In [ ]:
import logging

In [50]:
# Compose to srt
autelSubs:list[srt.Subtitle] = []
for a_sub in AutelSrtGenerator:
    # Try to extract the gps and camera data
    GPS_match = re.search(GPS_pattern,a_sub.content)
    Camera_match = re.search(Camera_pattern,a_sub.content)
    
    # If not match, continue.
    if not GPS_match or not Camera_match:
        logging.error(f"The format does not match. Contents:{a_sub.content}")
        continue        
    else:
        GPS_match_dict = GPS_match.groupdict()
        Camera_match_dict = Camera_match.groupdict()
        gps_lon=0.
        gps_lat=0.
        if GPS_match_dict["ew"] == "W":
            gps_lon = -float(GPS_match_dict["lon"])
        else:
            gps_lon = float(GPS_match_dict["lon"])
        if GPS_match_dict["ns"] == "S":
            gps_lat = -float(GPS_match_dict["lat"])
        else:
            gps_lat = float(GPS_match_dict["lat"])
        djiFormatMsg = DjiMini2SrtMessage(
            f_step=float(Camera_match_dict["f_num"]),
            ss=float(Camera_match_dict["shutter"]),
            iso=int(Camera_match_dict["iso"]),
            ev=float(Camera_match_dict["ev"]),
            gps_lon=gps_lon,
            gps_lat=gps_lat,
            altitude=float(GPS_match_dict["alt"])
            )
        # Replace content of the subtitle
        a_sub.content = djiFormatMsg.toSrtMessage()
        autelSubs.append(a_sub)
# print(srt.compose(autelSubs))
with open(AutelSrtPath.with_suffix(".a2d.srt"),"wt",encoding="utf-8") as fOut:
    fOut.write(srt.compose(autelSubs))